# Mod 1 Movie Project

#### Scrape the DVD / Blue Ray sales from https://www.the-numbers.com/
1. Import request to get the webpage in a json format 
1. Use beautiful soup to parse the html
2. scrape the top DVD sales for 2018 / 2019
3. Scrape the top blue ray sales for 2018 / 2019
4. Built function top_100_sales that intakes a URL of a top 100 page and outputs a list of dictionaries
5. Output example: 
    {'Name': 'Mamma Mia!',
     'rank': 100,
     'Units Sold': '232687',
     'Consumer Spending': '2522707'}

In [ ]:
import requests
import time

from bs4 import BeautifulSoup

import config


In [ ]:
#Movie Title corrections 
CORRECTION_DICT = {'Dr. Seussâ\x80\x99 The Grinch':'The Grinch',
                   'Spider-Man: Into The Spider-Verse 3D':'Spider-Man: Into the Spider-Verse',
                   'Nobodyâ\x80\x99s Fool':'Nobody’s Fool',
                   'Wonâ\x80\x99t You Be My Neighbor?':'Won’t You Be My Neighbor?',
                   'Oceanâ\x80\x99s 8':'Oceans 8'}

In [ ]:
#Scrapes the-numbers webpage for top 100 dvd / br sales 
def top_100_sales(url):
    top_list = []
    top_dict = {}
    
    top_url = requests.get(url)
    top_soup = BeautifulSoup(top_url.content, 'html.parser')
    top_numbers = top_soup.find_all('td', class_='data')
    
    

    for i in range(0, len(top_numbers), 3):
        rank = top_numbers[i].text 
        rank_int = int(rank)
        units_sold = top_numbers[i+1].text 
        consumer_spending = top_numbers[i+2].text
        movie_name = top_soup.find_all('b')[rank_int - 1].get_text()
        top_dict['Name'] = movie_name
        top_dict['rank'] = rank_int
        top_dict['Units Sold'] = units_sold.replace(',', '')
        consumer_spending =  consumer_spending.replace('$', '')
        top_dict['Consumer Spending'] = consumer_spending.replace(',', '')
        top_list.append(top_dict)
        top_dict = {}
        
    return top_list

In [ ]:
#corrects the incorrect movie titles 
def dvd_br_corrections(movie_list):
    new_movie_list = []
    
    for movie in movie_list: 
#         new_movie_list.append(CORRECTION_DICT.get(movie['Name'], movie['Name']))
        if movie['Name'] in CORRECTION_DICT.keys():
            movie['Name'] = CORRECTION_DICT[movie['Name']]

    return movie_list

# Top 2018 Domestic Movies

In [ ]:
#Takes in url of top 100 movies and a dictionary of the movies from the API Call 

def top_domestic_movies(url, final_movie_list):
    top_movies_list = []
    
    top_movies = requests.get(url)
    movies_soup = BeautifulSoup(top_movies.content, 'html.parser')
    movies_numbers = movies_soup.find_all('td', class_='data')
    
    
    for i in range(100):
        name = movies_soup.find_all('b')[i].get_text()
        top_movies_list.append(name)
    
    movies_info = movies_soup.find_all('td', align="right")
    
    movie_2018 = []
    dict_movie = {}
    rank = 0
    
    #in range 400 because each movie has a name, rank, domestic box office and opening weekend 

    for i in range(0, 400, 4):
        name = final_movie_list[rank]['original_title']
        rank += 1
        domestic_box_office = movies_info[i].text 
        opening_weekend = movies_info[i+2].text
        dict_movie['Name'] = name
        dict_movie['Rank'] = rank
        domestic_box_office = domestic_box_office.replace(',', '')
        dict_movie['Domestic Box Office'] = domestic_box_office.replace('$', '')
        opening_weekend = opening_weekend.replace(',', '')
        dict_movie['Opening Weekend'] = opening_weekend.replace('$', '')
        movie_2018.append(dict_movie)
        dict_movie = {}
        
    return movie_2018

In [ ]:
#creates tuple for dvd and br movies
def dvd_br_tuple(top_dvd_br_list):
    movie_tuple_list = []
    for i in top_dvd_br_list:
        movie_tuple = (i['Name'], i['rank'], i['Units Sold'], i['Consumer Spending'])
        movie_tuple_list.append(movie_tuple)
    return movie_tuple_list
        

In [ ]:
#creates tuple for top 100 movies 
def top_movie_tuple(top_movie_list):
    movie_tuple_list = []
    for i in top_movie_list:
        movie_tuple = (i['Name'], i['Rank'], i['Domestic Box Office'], i['Opening Weekend'])
        movie_tuple_list.append(movie_tuple)
    return movie_tuple_list

## Setting up the DB

In [ ]:
#import 
import mysql.connector 
from mysql.connector import errorcode
import config
import json

In [ ]:
#connecting to AWS DB instance
cnx = mysql.connector.connect(
        host = config.host,
        user = config.user,
        password = config.passwd)

print(cnx)
cursor = cnx.cursor()

db_name = 'Movies'

In [ ]:
#creates db
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [ ]:
cnx.commit()
cursor.close()
cnx.close()

In [ ]:
# CONNECT AGAIN FOR INSERTING DATA INTO TABLES
cnx = mysql.connector.connect(
        host = config.host,
        user = config.user,
        password = config.passwd,
        database = db_name
    )
cursor = cnx.cursor()
print(cnx)

#creating and commiting to the tables in DB 
def create_table_dvd18():
    
    create_dvd = """
    CREATE TABLE dvd_2018 (
    name TEXT NOT NULL,
    rank TEXT NOT NULL,
    units_sold TEXT,
    consumer_spending TEXT
    );
    """
    cursor.execute(create_dvd)
    
def commit_dvd18(dvd_2018_scrape):
    stmt = "INSERT INTO dvd_2018 (name, rank, units_sold, consumer_spending) VALUES (%s, %s, %s, %s)"
    cursor.executemany(stmt, dvd_2018_scrape)
    cnx.commit()
    
def create_table_dvd19():
    
    create_dvd_2019 = """
    CREATE TABLE dvd_2019 (
    name TEXT NOT NULL,
    rank TEXT NOT NULL,
    units_sold TEXT,
    consumer_spending TEXT
    );
    """
    cursor.execute(create_dvd_2019)
    
def commit_dvd19(dvd_2019_scrape):
    stmt = "INSERT INTO dvd_2019 (name, rank, units_sold, consumer_spending) VALUES (%s, %s, %s, %s)"
    cursor.executemany(dvd_2019_scrape)
    cnx.commit()
    
def create_table_br18():
    # Create a table for the br_2018
    create_br = """
    CREATE TABLE br_2018 (
    name TEXT NOT NULL,
    rank TEXT NOT NULL,
    units_sold TEXT,
    consumer_spending TEXT
    );
    """

    cursor.execute(create_br)
    
def commit_br18(br_2018_scrape):
    stmt = "INSERT INTO br_2018 (name, rank, units_sold, consumer_spending) VALUES (%s, %s, %s, %s)"
    cursor.executemany(stmt, br_2018_scrape)
    cnx.commit()
    
def create_table_br19():
    # Create a table for the br_2019
    create_br_2019 = """
    CREATE TABLE br_2019 (
    name TEXT NOT NULL,
    rank TEXT NOT NULL,
    units_sold TEXT,
    consumer_spending TEXT
    );
    """
    cursor.execute(create_br_2019)

def commit_br19(br_2019_scrape):
    stmt = "INSERT INTO br_2019 (name, rank, units_sold, consumer_spending) VALUES (%s, %s, %s, %s)"
    cursor.executemany(stmt, final_br_2019)
    cnx.commit()
    
def create_table_movies():
    create_top_movies_2018 = """
    CREATE TABLE top_movies_2018 (
    name TEXT NOT NULL,
    rank TEXT NOT NULL,
    Domestic_Box_Office TEXT,
    Opening_Weekend TEXT
    );
    """
    cursor.execute(create_top_movies_2018)
    
def commit_top_movies(top_movie_scrape):
    stmt = "INSERT INTO top_movies_2018 (name, rank, Domestic_Box_Office, Opening_Weekend) VALUES (%s, %s, %s, %s)"
    cursor.executemany(stmt, top_movie_scrape)
    cnx.commit()
    
cursor.close()
cnx.close()

# API Call 

In [ ]:
#API Key
KEY = config.api_key

In [ ]:
top_movies_100 = top_movies_list

# top_movies_100

In [ ]:
#helper function 
def movie_corrections(movie_list):
    new_movie_list = []
    
    for movie in movie_list: 
        new_movie_list.append(CORRECTION_DICT.get(movie, movie))

    return new_movie_list

In [ ]:
top_movies_100 = movie_corrections(top_movies_100)


In [ ]:
def movie_db_api_call(top_movies_100):
    count = 0
    new_movie_list_dict = []
    movie_dict = {}

    for movie in top_movies_100:

        movie = requests.get(f'https://api.themoviedb.org/3/search/movie?api_key={KEY}&query={movie}')
        movie_json = movie.json()
        # make 3 for loops
        # when counter = 39 sleep 10 seconds 

        #print(count, movie_json["results"][0]["id"], movie_json["results"][0]["original_title"], movie_json["results"][0]["vote_average"])
        movie_id = movie_json["results"][0]["id"]
        movie_info = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={KEY}&language=en÷-US')
        movie_data = movie_info.json()
        new_movie_list_dict.append(movie_data)
        time.sleep(1.3)
        
    return new_movie_list_dict


In [ ]:
def movie_dicts_final(new_movie_list_dict):
    count = 0
    final_movie_list = []
    movie_dict = {}
    for i in range(len(new_movie_list_dict)):
        m = new_movie_list_dict
        movie_dict['id'] = m[i]['id']
        movie_dict['original_title'] = m[i]['original_title']
        movie_dict['budget'] = m[i]['budget']
        movie_dict['vote_average'] = m[i]["vote_average"]
        movie_dict['revenue'] = m[i]['revenue']
        movie_dict['release_date'] = m[i]['release_date']
        movie_dict['runtime'] = m[i]['runtime']
        genres = m[i]['genres']
        for i in range(len(genres)):
            movie_dict[f'genre_{i}'] = genres[i]['name']
        final_movie_list.append(movie_dict)
        movie_dict = {}
        
    return final_movie_list
    

In [ ]:
def final_movie_tuple(final_movie_list):    
    count = 0
    final_movie_list_2018 = []
    for n in range(len(final_movie_list)):
        i = final_movie_list[n]

        if len(i) == 8:
            movie_tuple = (i['id'], i['original_title'], i['budget'], i['vote_average'], 
                           i['revenue'], i['release_date'], i['runtime'], i['genre_0'])
            stmt = "INSERT INTO top_movies_info (id, original_title, budget, vote_average, revenue, release_date, runtime, genre) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
            cursor.execute(stmt, movie_tuple)

        elif len(i) == 9:
            movie_tuple = (i['id'], i['original_title'], i['budget'], i['vote_average'], 
                           i['revenue'], i['release_date'], i['runtime'], i['genre_0'], 
                           i['genre_1'])
            stmt = "INSERT INTO top_movies_info (id, original_title, budget, vote_average, revenue, release_date, runtime, genre, genre_1) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
            cursor.execute(stmt, movie_tuple)

        elif len(i) == 10:
            movie_tuple = (i['id'], i['original_title'], i['budget'], i['vote_average'], 
                           i['revenue'], i['release_date'], i['runtime'], i['genre_0'], 
                           i['genre_1'], i['genre_2'])
            stmt = "INSERT INTO top_movies_info (id, original_title, budget, vote_average, revenue, release_date, runtime, genre, genre_1, genre_2) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
            cursor.execute(stmt, movie_tuple)

        elif len(i) == 11:
            movie_tuple = (i['id'], i['original_title'], i['budget'], i['vote_average'], 
                           i['revenue'], i['release_date'], i['runtime'], i['genre_0'], 
                           i['genre_1'], i['genre_2'], i['genre_3'])
            stmt = "INSERT INTO top_movies_info (id, original_title, budget, vote_average, revenue, release_date, runtime, genre, genre_1, genre_2, genre_3) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
            cursor.execute(stmt, movie_tuple)

        elif len(i) == 12:
            movie_tuple = (i['id'], i['original_title'], i['budget'], i['vote_average'], 
                           i['revenue'], i['release_date'], i['runtime'], i['genre_0'], 
                           i['genre_1'], i['genre_2'], i['genre_3'], i['genre_4'])
            stmt = "INSERT INTO top_movies_info (id, original_title, budget, vote_average, revenue, release_date, runtime, genre, genre_1, genre_2, genre_3, genre_4) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
            cursor.execute(stmt, movie_tuple)

        elif len(i) == 13:
            movie_tuple = (i['id'], i['original_title'], i['budget'], i['vote_average'], 
                           i['revenue'], i['release_date'], i['runtime'], i['genre_0'], 
                           i['genre_1'], i['genre_2'], i['genre_3'], i['genre_4'], i['genre_5'])
            stmt = "INSERT INTO top_movies_info (id, original_title, budget, vote_average, revenue, release_date, runtime, genre, genre_1, genre_2, genre_3, genre_4, genre_5) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
            cursor.execute(stmt, movie_tuple)

        else: 
            movie_tuple = (i['id'], i['original_title'], i['budget'], i['vote_average'], 
                           i['revenue'], i['release_date'], i['runtime'], i['genre_0'], 
                           i['genre_1'], i['genre_2'], i['genre_3'], i['genre_4'], i['genre_5'], i['genre_6'])
            stmt = "INSERT INTO top_movies_info (id, original_title, budget, vote_average, revenue, release_date, runtime, genre, genre_1, genre_2, genre_3, genre_4, genre_5, genre_6) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
            cursor.execute(stmt, movie_tuple)

        print(count)
    #     print(f'movie_tuple = {movie_tuple}')
        count += 1
        final_movie_list_2018.append(movie_tuple)
        return final_movie_list_2018

# Adding The Movie DB API to SQL

In [ ]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.passwd,
    database = db_name
)
cursor = cnx.cursor()
print(cnx)

In [ ]:
# (i['id'], i['original_title'], i['budget'], i['vote_average'], 
#                        i['revenue'], i['release_date'], i['runtime'], i['genre_0'], 
#                        i['genre_1'], i['genre_2'], i['genre_3'], i['genre_4'], i['genre_5'], i['genre_6'])
#creating table:
# Create a table for the top_movies_2018
create_top_movies_info = """
CREATE TABLE top_movies_info (
id TEXT NOT NULL,
original_title TEXT NOT NULL,
budget TEXT NOT NULL,
vote_average TEXT,
revenue TEXT,
release_date DATETIME, 
runtime REAL,
genre TEXT,
genre_1 TEXT,
genre_2 TEXT,
genre_3 TEXT,
genre_4 TEXT,
genre_5 TEXT,
genre_6 TEXT
);
"""

In [ ]:
cursor.execute(create_top_movies_info)

In [ ]:
cnx.commit()

In [ ]:
cursor.close()
cnx.close()